**Задание 1**

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

    'Центр': ['москва', 'тула', 'ярославль'],

    'Северо-Запад': ['петербург', 'псков', 'мурманск'],

    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

Результат классификации запишите в отдельный столбец region.

In [1]:
geo_data = {
    'Центр': ['москва', 'тула', 'ярославль'],
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

In [2]:
import pandas as pd

In [3]:
keywords = pd.read_csv('keywords.csv')
keywords.head(5)

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [4]:
def get_geo(row, geo_data):
    '''
    Вернуть данные по геолокации при их наличии в тексте. В противном случае - неопределенное положение
    '''
    for geo, items in geo_data.items():
        for item in items:
            if item in row['keyword']:
                return geo
    
    return 'undefined'

In [5]:
%%time
# Как сделать побыстрее ?

keywords['region'] = keywords.apply(get_geo, geo_data=geo_data, axis=1)
keywords.head(5)

CPU times: user 30.7 s, sys: 32.5 ms, total: 30.7 s
Wall time: 30.9 s


In [6]:
keywords.groupby('region').count()

,keyword,shows
region,,
undefined,99253,99253
Дальний Восток,94,94
Северо-Запад,266,266
Центр,387,387


**Задание 2**

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
    - оценка 2 и меньше - низкий рейтинг
    - оценка 4 и меньше - средний рейтинг
    - оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class


In [7]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head(10)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [8]:
def get_rating(row):
    '''
    Классифицировать рейтинг в зависимости от уровня рейтинга
    '''
    if row['rating'] <= 2:
        return 'Низкий рейтинг'
    elif row['rating'] <= 4:
        return 'Средний рейтинг'
    else:
        return 'Высокий рейтинг'

In [9]:
ratings['rating_level'] = ratings.apply(get_rating, axis=1)
ratings.head(10)

,userId,movieId,rating,timestamp,rating_level
0,1,31,2.5,1260759144,Средний рейтинг
1,1,1029,3.0,1260759179,Средний рейтинг
2,1,1061,3.0,1260759182,Средний рейтинг
3,1,1129,2.0,1260759185,Низкий рейтинг
4,1,1172,4.0,1260759205,Средний рейтинг
5,1,1263,2.0,1260759151,Низкий рейтинг
6,1,1287,2.0,1260759187,Низкий рейтинг
7,1,1293,2.0,1260759148,Низкий рейтинг
8,1,1339,3.5,1260759125,Средний рейтинг
9,1,1343,2.0,1260759131,Низкий рейтинг


**Задание 3**

Посчитайте среднее значение Lifetime киноманов (пользователи, которые поставили 100 и более рейтингов). 
Под Lifetime понимается разница между максимальным и минимальным значением timestamp для каждого пользователя. 
Ответ дайте в днях.


In [10]:
# получаем список userId для киноманов (>= 100 рейтингов)
userid_more_100 = ratings.groupby('userId').count()[['rating']][ratings.groupby('userId').count()[['rating']]['rating'] >= 100]
userid_more_100_list = userid_more_100.reset_index()['userId'].tolist()

In [11]:
# формируем из этого списка отдельный data frame
updated_ratings = ratings[ ratings['userId'].isin(userid_more_100_list) ]
updated_ratings.head()

,userId,movieId,rating,timestamp,rating_level
147,4,10,4.0,949810645,Средний рейтинг
148,4,34,5.0,949919556,Высокий рейтинг
149,4,112,5.0,949810582,Высокий рейтинг
150,4,141,5.0,949919681,Высокий рейтинг
151,4,153,4.0,949811346,Средний рейтинг


In [12]:
# считаем минимальный и максимальный timestamp
if ('min_timestamp' not in updated_ratings.columns and 'max_timestamp' not in updated_ratings.columns):    
    grouped_ratings = updated_ratings.groupby('userId').agg(['min', 'max'])['timestamp']
    grouped_ratings.columns = ['min_timestamp', 'max_timestamp']
    updated_ratings = updated_ratings.merge(grouped_ratings, on='userId', how='left')

# формируем значение lifetime 
updated_ratings['lifetime'] = updated_ratings['max_timestamp'] - updated_ratings['min_timestamp']

# высчитываем среднее значение для всех пользователей
average_lifetime = updated_ratings['lifetime'].mean() / 60 / 60 / 24
print('Среднее значение Lifetime для киноманов в днях : {}'.format(average_lifetime))

Среднее значение Lifetime для киноманов в днях : 1053.3271445414307


**Задание 4**

Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из материалов занятия. Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
    - для каждой строки пройдите по всем годам списка years
    - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
    - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по убыванию рейтинга


In [13]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head(5)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [14]:
movies = pd.read_csv('ml-latest-small/movies.csv')
movies.head(5)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [15]:
if 'title' not in ratings.columns:
    ratings = ratings.merge(movies, on='movieId', how='left')
ratings.head(5)

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama


In [16]:
years = [x for x in range(1950, 2011)]
print(years)

[1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010]


In [17]:
def production_year(row):
    '''
    Получить значение production_year
    '''
    for year in years:
        if str(year) in row['title']:
            return year
    return 1900

In [18]:
%%time
# too long
ratings['year'] = ratings.apply(production_year, axis=1)

CPU times: user 2min 59s, sys: 151 ms, total: 2min 59s
Wall time: 3min


In [19]:
ratings.groupby('year').mean().sort_values('rating', ascending=False).head(100)

,userId,movieId,rating,timestamp
year,,,,
1957,360.933544,2750.965190,4.014241,1.083707e+09
1972,359.694878,3983.538976,4.011136,1.122759e+09
1952,346.394737,4107.796053,4.000000,1.090512e+09
1954,358.228324,2867.661850,3.994220,1.070591e+09
1951,347.106996,2605.588477,3.983539,1.052714e+09
1974,357.181481,3956.801852,3.978704,1.112526e+09
1962,350.671196,3479.100543,3.952446,1.094200e+09
1950,355.720339,3503.427966,3.915254,1.090648e+09
1977,351.040059,3604.516320,3.905786,1.116007e+09


Судя по среднему рейтингу фильмов, утверждение о том, что с каждым годом фильмы становятся хуже, верно